In [31]:
import re
import torch
import random
import numpy as np
import tensorflow as tf

In [32]:
STRINGS_LIST_LEN = 10000

In [33]:
alphabet = ['a', 'b', 'c', 'd']
pattern = 'abcda'

In [34]:
def insert_pattern(string: str):
    index = random.randint(0, 10)
    string = string[:index] + pattern + string[index + len(pattern):]
    return string
out = insert_pattern('aaaaaaaaaaaaaaa')
print(out, len(out))

aaaaaaaaaaabcda 15


In [35]:
strings = []
labels = []
for i in range(STRINGS_LIST_LEN):
    strings.append(''.join(random.choices(alphabet, k=15)))
    if i % 2 == 0 and not re.search(pattern, strings[i]):
        strings[i] = insert_pattern(strings[i])
    if i % 2 == 1 and re.search(pattern, strings[i]):
        strings[i] = re.sub(pattern, ''.join(random.choices(alphabet, k=5)), strings[i], 1)
        while re.search(pattern, strings[i]):
            strings[i] = re.sub(pattern, ''.join(random.choices(alphabet, k=5)), strings[i], 1)
    labels.append((i+1)%2)
    # if i % 2 == 0: labels.append(1)
    # else: labels.append(0)

In [36]:
temp = list(zip(strings, labels))
random.shuffle(temp)
strings, labels = zip(*temp)
strings, labels = list(strings), list(labels)

for i in range(10):   
    print(strings[i], labels[i])

ddbaabaacdbbcca 0
bddcdabcbcbcacb 0
abaabcdaabbdbcb 1
dadccaabdcbdcca 0
bccbadcddacdbdb 0
dbcaadabcdacdcb 1
bcbbabcdabbbadb 1
dcbcadbdaaaacbb 0
badabbaaaabcdad 1
cbbacddcaabcdab 1


In [37]:
hot_strings = [tf.keras.utils.to_categorical(np.array([alphabet.index(letter) for letter in string]), num_classes=len(alphabet)) for string in strings]

In [38]:
train_strings = np.array(hot_strings[:8000])
train_labels = np.array(labels[:8000])

test_strings = np.array(hot_strings[8000:])
test_labels = np.array(labels[8000:])
print(train_strings[0])
print(train_labels[0])

[[0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]]
0


In [39]:
model_1 = tf.keras.models.Sequential()
model_1.add(tf.keras.layers.Conv1D(filters=1, kernel_size=5, activation='relu', input_shape=(15, 4)))
model_1.add(tf.keras.layers.Flatten())
# model_1.add(tf.keras.layers.Dense(10, activation='sigmoid'))
model_1.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model_1.summary()
model_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 11, 1)             21        
                                                                 
 flatten_3 (Flatten)         (None, 11)                0         
                                                                 
 dense_4 (Dense)             (None, 1)                 12        
                                                                 
Total params: 33
Trainable params: 33
Non-trainable params: 0
_________________________________________________________________


In [40]:

# Train the model_1
model_1.fit(train_strings, train_labels, epochs=10, batch_size=32, validation_data=(test_strings, test_labels))


Epoch 1/10
250/250 [==============================] - 1s 3ms/step - loss: 0.6945 - accuracy: 0.5368 - val_loss: 0.6719 - val_accuracy: 0.5650
Epoch 2/10
250/250 [==============================] - 1s 2ms/step - loss: 0.6616 - accuracy: 0.6109 - val_loss: 0.6462 - val_accuracy: 0.6500
Epoch 3/10
250/250 [==============================] - 1s 2ms/step - loss: 0.6406 - accuracy: 0.6635 - val_loss: 0.6266 - val_accuracy: 0.6795
Epoch 4/10
250/250 [==============================] - 1s 2ms/step - loss: 0.6207 - accuracy: 0.6879 - val_loss: 0.6070 - val_accuracy: 0.6890
Epoch 5/10
250/250 [==============================] - 1s 2ms/step - loss: 0.6016 - accuracy: 0.6977 - val_loss: 0.5895 - val_accuracy: 0.7090
Epoch 6/10
250/250 [==============================] - 1s 2ms/step - loss: 0.5859 - accuracy: 0.7081 - val_loss: 0.5759 - val_accuracy: 0.7180
Epoch 7/10
250/250 [==============================] - 1s 2ms/step - loss: 0.5733 - accuracy: 0.7175 - val_loss: 0.5648 - val_accuracy: 0.7215
Epoch 

In [41]:
filter = model_1.layers[0].get_weights()[0][:, :, 0]
print(filter)

[[ 0.2549068   0.25266325  0.2580355  -0.33550078]
 [ 0.10440952  0.07230814 -0.19184707  0.07905141]
 [-0.89782286  0.08461267 -0.0964635   0.05429755]
 [-0.56495476  0.0907548   0.06626719  0.73877513]
 [-0.8935974   0.2716938   0.5259349  -0.43592164]]


In [42]:
loss, accuracy = model_1.evaluate(test_strings, test_labels)
print(f'loss: {loss}, accuracy: {accuracy}')

63/63 [==============================] - 0s 1ms/step - loss: 0.5414 - accuracy: 0.7275
loss: 0.5413827896118164, accuracy: 0.7275000214576721


In [43]:
your_str = ['aaaaaaaaaaaaaaa', # 0
            'bbbbbbbbbbbbbbb', # 0
            'ccccccccccccccc', # 0
            'ddddddddddddddd', # 0
            'abcdadbcabdaddc', # 1
            'acbacbdacbdbcda', # 0
            'adcbadcbabcdabc'] # 1
your_hot_str = np.array([tf.keras.utils.to_categorical(np.array([alphabet.index(letter) for letter in string[:15]]), num_classes=len(alphabet)) for string in your_str])
# print(your_hot_str[0], len(your_hot_str))

result = model_1.predict([your_hot_str])
print(result)

1/1 [==============================] - 0s 70ms/step
[[0.7540543 ]
 [0.02042863]
 [0.08621658]
 [0.72168124]
 [0.52918416]
 [0.362359  ]
 [0.53752214]]


Model with one filter: accuracy varies between 0.7 and 0.94


Sigmoid activation function got much better results than activation relu function

```[[-0.9766118   0.8987984  -1.0841043  -1.103324  ]
 [-1.1577314  -1.0720665   0.8588849  -1.1252344 ]
 [-1.4096966  -1.2763376  -1.2028228   0.6865602 ]
 [ 0.5466621  -1.4944448  -1.5552505  -1.4274256 ]
 [-0.36988205 -0.25175053 -0.46649975 -0.4346084 ]]
```
 loss: 0.21468232572078705, accuracy: 0.9419999718666077

In [104]:
model_2 = tf.keras.models.Sequential()
model_2.add(tf.keras.layers.Conv1D(filters=2, kernel_size=5, activation='relu', input_shape=(15, 4)))
model_2.add(tf.keras.layers.MaxPool1D(pool_size=2, strides=1, padding='same'))
model_2.add(tf.keras.layers.Conv1D(filters=2, kernel_size=5, activation='relu', input_shape=(15, 4)))
# model_2.add(tf.keras.layers.MaxPool1D(pool_size=2, strides=1, padding='same'))
model_2.add(tf.keras.layers.Flatten())
model_2.add(tf.keras.layers.Dense(10, activation='sigmoid'))
model_2.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model_2.summary()
model_2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_20 (Conv1D)          (None, 11, 2)             42        
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 11, 2)            0         
 1D)                                                             
                                                                 
 conv1d_21 (Conv1D)          (None, 7, 2)              22        
                                                                 
 flatten_13 (Flatten)        (None, 14)                0         
                                                                 
 dense_23 (Dense)            (None, 10)                150       
                                                                 
 dense_24 (Dense)            (None, 1)                 11        
                                                     

In [105]:
# Train the model_2 
model_2.fit(train_strings, train_labels, epochs=10, batch_size=32, validation_data=(test_strings, test_labels))

Epoch 1/10
250/250 [==============================] - 2s 4ms/step - loss: 0.7214 - accuracy: 0.5014 - val_loss: 0.6936 - val_accuracy: 0.4945
Epoch 2/10
250/250 [==============================] - 1s 3ms/step - loss: 0.6767 - accuracy: 0.5465 - val_loss: 0.6337 - val_accuracy: 0.7025
Epoch 3/10
250/250 [==============================] - 1s 3ms/step - loss: 0.5829 - accuracy: 0.7387 - val_loss: 0.5547 - val_accuracy: 0.7480
Epoch 4/10
250/250 [==============================] - 1s 3ms/step - loss: 0.5224 - accuracy: 0.7676 - val_loss: 0.5015 - val_accuracy: 0.7785
Epoch 5/10
250/250 [==============================] - 1s 3ms/step - loss: 0.4685 - accuracy: 0.7961 - val_loss: 0.4529 - val_accuracy: 0.8090
Epoch 6/10
250/250 [==============================] - 1s 3ms/step - loss: 0.4190 - accuracy: 0.8263 - val_loss: 0.4029 - val_accuracy: 0.8395
Epoch 7/10
250/250 [==============================] - 1s 3ms/step - loss: 0.3687 - accuracy: 0.8614 - val_loss: 0.3542 - val_accuracy: 0.8710
Epoch 

In [106]:
# filter1 = model_2.layers[0].get_weights()[0][:, :, 0]
# filter2 = model_2.layers[0].get_weights()[1][:, :, 0]
# print(filter1, filter2)

In [107]:
loss_2, accuracy_2 = model_2.evaluate(test_strings, test_labels)
print(f'loss: {loss_2}, accuracy: {accuracy_2}')

63/63 [==============================] - 0s 2ms/step - loss: 0.2502 - accuracy: 0.9180
loss: 0.25019821524620056, accuracy: 0.9179999828338623


In [108]:
your_str = ['aaaaaaaaaaaaaaa', # 0
            'bbbbbbbbbbbbbbb', # 0
            'ccccccccccccccc', # 0
            'ddddddddddddddd', # 0
            'abcdadbcabdaddc', # 1
            'acbacbdacbdbcda', # 0
            'adcbadcbabcdabc'] # 1
your_hot_str = np.array([tf.keras.utils.to_categorical(np.array([alphabet.index(letter) for letter in string[:15]]), num_classes=len(alphabet)) for string in your_str])
# print(your_hot_str[0], len(your_hot_str))

result = model_2.predict([your_hot_str])
print(result)

1/1 [==============================] - 0s 82ms/step
[[0.7497108 ]
 [0.66065943]
 [0.7497108 ]
 [0.05385881]
 [0.3298482 ]
 [0.13850188]
 [0.8188818 ]]


In [109]:
# from torch.autograd import Variable
# class Model_1_PT(torch.nn.Module):
#     def __init__(self) -> None:
#         super().__init__()
#         self.model = torch.nn.Sequential(
#             torch.nn.Conv1d(15, 11, 4),
#             torch.nn.ReLU(),
#             torch.nn.Flatten(),
#             torch.nn.Linear(11, 1)
#         )
#         self.double()
    
#     def forward(self, input):
#         return self.model(input)
    
#     def train(self, train_strings, train_labels):
#         loss_function = torch.nn.BCELoss()
#         optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
#         # accuracy=tf.keras.metrics.BinaryAccuracy()
#         for epoch in range(100):
#             # print('wpadlo1')
#             output = self.forward(train_strings)
#             # print(output, train_labels)
#             loss = loss_function(output, train_labels)
#             # print('wpadlo')
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
#             # accuracy(output, train_labels)
    
# model_1_pt = Model_1_PT().float()

# model_1_pt.forward(torch.tensor(np.array(train_strings)))

# print(model_1_pt)
# # model_1_pt.forward(torch.tensor(train_strings))
# # train_labels_r = []
# # for index, value in enumerate(train_labels):
# #     # print(index)
# #     train_labels_r.append([float(value)])
# # print('torch.empty(3).random_(2)', torch.empty(3).random_(2))
# train_labels_r = np.array(train_labels)
# train_labels_r=np.reshape(train_labels_r, (8000,1))
# train_labels_r = Variable(torch.from_numpy(train_labels_r)).float()
# train_strings_V2 = Variable(torch.from_numpy(train_strings)).float()
# train_strings_V1 = torch.tensor(np.array(train_strings)).float()
# model_1_pt.train(train_strings_V1, train_labels_r)

# output = model_1_pt.forward(torch.tensor(np.array(test_strings)))
# print(output)
# print(test_labels)